In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
plt.gcf().subplots_adjust(bottom=0.15)

In C:\Users\Titus\Anaconda3\envs\QF3.8\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Titus\Anaconda3\envs\QF3.8\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Titus\Anaconda3\envs\QF3.8\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Titus\Anaconda3\envs\QF3.8\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Titus\Anaconda3\envs\QF3.8\lib\site-

<Figure size 432x288 with 0 Axes>

In [2]:
data_raw = pd.read_csv("Data/dataset_mood_smartphone.csv")

In [3]:
print(data_raw.columns)

Index(['Unnamed: 0', 'id', 'time', 'variable', 'value'], dtype='object')


In [4]:
variables = data_raw['variable'].unique()
print(variables)

['mood' 'circumplex.arousal' 'circumplex.valence' 'activity' 'screen'
 'call' 'sms' 'appCat.builtin' 'appCat.communication'
 'appCat.entertainment' 'appCat.finance' 'appCat.game' 'appCat.office'
 'appCat.other' 'appCat.social' 'appCat.travel' 'appCat.unknown'
 'appCat.utilities' 'appCat.weather']


In [5]:
print(len(data_raw))

376912


In [6]:
data_raw['time'] = pd.to_datetime(data_raw['time'])

In [7]:
data_raw = data_raw.drop(columns=['Unnamed: 0'])



In [8]:
#set negative values of duration to zero\

data_raw = data_raw[~((data_raw['variable']=='appCat.builtin') & (data_raw['value'] < 0))]

In [9]:
df_wide = data_raw.pivot_table(index=['id', 'time'], columns='variable', values='value').reset_index()

In [19]:
# import plotly.graph_objects as go
# df_temp = df_wide.copy()
# df_temp['date'] = df_temp['time'].dt.date
# df_mood = df_temp.groupby(['id', 'date'])['mood'].mean().unstack(level='date').sort_index(level='date', axis=1)
#
# fig = go.Figure(data=go.Heatmap(z=df_mood.values,
#                                 x=df_mood.columns,
#                                 y=df_mood.index.values,
#                                 colorscale='rdylgn',
#                                 colorbar=dict(title='Mood'),
#                                 zmid=5.5
#                                 ))
#
# fig.update_layout(
#         title="Mood Timeline",
#         xaxis_title="Date",
#         yaxis_title="Respondent"
#     )
#
# fig.show()
# fig.write_html("plots/mood_heatmap_raw.html")

In [ ]:
def create_affect_feature(row):
    if row['circumplex.valence'] == 'NaN' or row['circumplex.arousal'] == 'NaN':
        return
    else:
        return (row['circumplex.arousal']*5)+row['circumplex.valence']
        

In [ ]:
df_wide['affect'] = df_wide.apply(lambda row: create_affect_feature(row), axis=1)

In [ ]:
display(df_wide)

In [ ]:
#remove first part for AS14.01 as it contains only few days of mood reports
to_remove = (df_wide['id'] == 'AS14.01') & (df_wide['time'] < pd.Timestamp(2014, 3, 21))
df_wide = df_wide.loc[~to_remove]

In [ ]:
display(df_wide)

In [ ]:
#filter data to timestamps where mood was registered
df_wide_relevant = pd.DataFrame()
for idx in df_wide['id'].unique():
    df_id = df_wide.loc[df_wide['id']==idx]
    df_id_notnull = df_id.loc[df_id['mood'].notnull()]
    start = df_id_notnull['time'].min()
    end = df_id_notnull['time'].max()
    df_id_relevant = df_id.loc[(df_id['time'] >= start) & (df_id['time'] <= end)]
    df_wide_relevant = df_wide_relevant.append(df_id_relevant)



In [ ]:
display(df_wide_relevant)

In [ ]:
df_screentime = df_wide_relevant[~df_wide_relevant['screen'].isna()]


In [ ]:
num_of_days_per_user = {}
for user in df_screentime['id'].unique():
    num_of_days_per_user[user] = len(df_screentime[df_screentime['id']==user]['time'].dt.normalize().value_counts())
    
print(num_of_days_per_user)

In [ ]:
def get_sleep(df):
    sleep_durations = []
    dates = []
    user_list = []
    for user in df['id'].unique():
        new_user = True
        prev_row = None
        for idx, row in df[df['id']==user].iterrows():
            if new_user:
                prev_row = row
                new_user = False
                continue
    
            if (row['time'] - prev_row['time']).total_seconds() > 240*60:
                if (row['time'].hour < 12) and row['time'].hour > 4:
                    dates.append(row['time'])
                    sleep_durations.append((row['time'] - prev_row['time']).seconds / 3600)
                    user_list.append(user)
            prev_row = row
        
            
    dates = pd.Series(dates)
    sleep_durations = pd.Series(sleep_durations)
    users = pd.Series(user_list)
    sleep_df = pd.DataFrame({'time': dates, 'sleep': sleep_durations, 'id':users})
    return sleep_df

In [ ]:
user_sleep = get_sleep(df_screentime)

In [ ]:
# Shift sleep time to put it in the prediction day
from datetime import timedelta
user_sleep['time'] = user_sleep['time'] - timedelta(days=1)

In [ ]:
temporary = df_wide_relevant.copy()
df_wide_relevant = pd.concat([df_wide_relevant, user_sleep], sort=False)

### Exploration

In [ ]:
reported_days = []
for idx in df_wide_relevant['id'].unique():
    df_id = df_wide_relevant.loc[df_wide_relevant['id']==idx]
    df_id = df_id.loc[df_id['mood'].notnull()]
#     df_id.plot(x='time', y=('mood', 'mean'), title=idx, xlim=(df_wide_relevant['time'].min(), df_wide_relevant['time'].max()))
    num_days = (df_id['time'].max() - df_id['time'].min()).days
    reported_days.append(num_days)




In [ ]:
# num reported days per id
plt.bar(x=df_wide_relevant['id'].unique(), height=reported_days, width=0.5)
# plt.scatter(*zip(*reported_days))
plt.xticks(rotation='vertical')
plt.xlabel('Respondent id')
plt.ylabel('Days reported')
plt.tight_layout()
plt.savefig('ReportedDays.pdf')
plt.show()


In [ ]:
reported_days_per_week = df_wide_relevant.groupby(pd.Grouper(key='time', freq='W'))['mood'].count()
print(reported_days_per_week)

In [ ]:
df_daily = df_wide_relevant.groupby([pd.Grouper(key='time', freq='D'), 'id']).agg({'mood': ['min', 'max', 'median', 'mean', 'std', 'count'],
                                                                           'circumplex.arousal': ['min', 'max', 'median', 'mean', 'std', 'count'],
                                                                           'circumplex.valence': ['min', 'max', 'median', 'mean', 'std', 'count'],
                                                                           'affect': ['min', 'max', 'median', 'mean', 'std', 'count'],
                                                                           'activity': ['min', 'max', 'median', 'mean', 'std', 'count'],
                                                                           'screen': ['min', 'max', 'median', 'mean', 'std', 'count', 'sum'],
                                                                           'call':'count',
                                                                           'sms':'count',
                                                                           'appCat.builtin':['min', 'max', 'median', 'mean', 'std', 'count', 'sum'],
                                                                           'appCat.communication':['min', 'max', 'median', 'mean', 'std', 'count', 'sum'],
                                                                           'appCat.entertainment':['min', 'max', 'median', 'mean', 'std', 'count', 'sum'],
                                                                           'appCat.finance':['min', 'max', 'median', 'mean', 'std', 'count', 'sum'],
                                                                           'appCat.game':['min', 'max', 'median', 'mean', 'std', 'count', 'sum'],
                                                                           'appCat.office':['min', 'max', 'median', 'mean', 'std', 'count', 'sum'],
                                                                           'appCat.other':['min', 'max', 'median', 'mean', 'std', 'count', 'sum'],
                                                                           'appCat.social':['min', 'max', 'median', 'mean', 'std', 'count', 'sum'],
                                                                           'appCat.travel':['min', 'max', 'median', 'mean', 'std', 'count', 'sum'],
                                                                           'appCat.unknown':['min', 'max', 'median', 'mean', 'std', 'count', 'sum'],
                                                                           'appCat.utilities':['min', 'max', 'median', 'mean', 'std', 'count', 'sum'],
                                                                           'appCat.weather':['min', 'max', 'median', 'mean', 'std', 'count', 'sum'],
                                                                           'sleep':['mean']  # has 1 observation per day & mean keeps the NaN's
                                                                           }).reset_index()

In [ ]:
display(df_daily)

In [ ]:
average_mood = df_daily.groupby([pd.Grouper(key='time', freq='W'), 'id'])[[('mood','mean')]].mean().reset_index()
print(average_mood.reset_index())
dates = pd.date_range(start='2014-03-02', end='2014-06-15', freq='W')
average_mood.plot(legend=False, ylim=[4,8], xticks=dates, xlabel='Date', ylabel='Average mood')

# test =average_mood.pivot(index=('time',''), columns='id', values=('mood','mean'))
# print(test)
# test.plot()

In [ ]:
for idx in df_daily['id'].unique():
    df_id = df_daily.loc[df_daily['id']==idx]
    df_id.plot(x='time', y=('mood', 'mean'), title=idx, xlim=(df_wide_relevant['time'].min(), df_wide_relevant['time'].max()))

In [ ]:
 ## Fill Nan's for zero columns
cols_count = ['call', 'sms']
cols_time = list(filter(lambda col: col.startswith('appCat.'), variables.tolist())) + ['screen']

df_filled = df_daily[cols_count+cols_time].fillna(0) # Workaround due to MultiIndex on columns fucking up
df_daily.fillna(df_filled, inplace=True)

display(df_daily)

In [ ]:
## Forward Fill Nan's for score columns
cols_score = ['mood', 'circumplex.arousal', 'circumplex.valence', 'affect', 'activity']

df_temp = df_daily.set_index(['id', 'time']).sort_index()
df_filled = df_temp[cols_score].groupby('id').ffill(limit=7)  # max allowed gap of .. days

df_daily = df_temp.fillna(df_filled).reset_index()

display(df_daily)


In [ ]:
# Fill NaN-sleep at average sleep per user_id

#df_temp = df_daily.set_index('id', 'time').sort_index()
#usr_avg_sleep = df_temp[('sleep', 'mean')].groupby(by='id').expanding().mean() # poging tot expanding mean

df_temp = df_daily.set_index('id').sort_index()
usr_avg_sleep = df_temp.groupby(by='id').agg({('sleep', 'mean'): 'mean'})
df_temp2 = df_temp.join(usr_avg_sleep, rsuffix='_avg')
df_temp2[('sleep', 'mean')] = df_temp2[('sleep', 'mean')].fillna(df_temp2[('sleep_avg', 'mean')])
df_daily = df_temp2.reset_index()

In [ ]:
## Day of Week dummy variables
# TODO: rekening houden met Dummy Variable trap voor linear or polynomial regressions! => OneHotEncoding
df_daily[['mon', 'thue', 'wed', 'thu', 'fri', 'sat', 'sun']] = pd.get_dummies(df_daily['time'].dt.dayofweek)

display(df_daily.iloc[:, -7:])

df_daily.to_pickle('data_clean_daily.pkl')

In [ ]:
## Tabular aggregation
df_temp = df_daily.copy().set_index(['id', 'time']).sort_index()

lookback_days = 7
df_tab = df_temp[cols_score+cols_count+cols_time].rolling(window=lookback_days).mean()
df_tab = pd.concat([df_tab, df_temp[['mon', 'thue', 'wed', 'thu', 'fri', 'sat', 'sun']]], axis=1)   # DayOfWeek excluded from rolling mean
df_tab['target'] = df_temp[('mood', 'mean')].shift(periods=-1) # next day mean mood

display(df_tab)

In [ ]:
## TODO: drop nan rows? or don't use burn-in period
df_tab.dropna(axis=0, inplace=True)


In [ ]:
# dummy model for all users together
df_data = df_tab.reset_index()
df_data.to_pickle('data_clean.pkl')


In [ ]:
df_data.drop('time', axis=1, inplace=True)
X = df_data.iloc[:, :-1].values
y = df_data.iloc[:, -1].values.reshape(-1, 1)
print(X)
print(y)

In [ ]:
# Encode user_id
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = ct.fit_transform(X)
print(X)

In [ ]:
# Data split
# TODO: should use rolling windows actually
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# print(X_train)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.18, random_state=0)

In [ ]:
min_dimension = min(*X_train.shape, *X_val.shape, *X_test.shape)

In [ ]:
# Feature Scaling
# from sklearn.preprocessing import StandardScaler
# sc_X = StandardScaler()
# sc_y = StandardScaler()

# # TODO: Don't scale the categories
# X_train = sc_X.fit_transform(X_train)
# X_test = sc_X.transform(X_test)
# y_train = sc_y.fit_transform(y_train)


In [ ]:
# PCA
from sklearn.decomposition import PCA
pca = PCA(n_components = min_dimension)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)


In [ ]:
# Training the Support Vector Regression model on the Training set
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
# regressor = SVR(kernel = 'poly')

regressor.fit(X_train, y_train.ravel())

In [ ]:
y_pred = regressor.predict(X_test)
# y_pred = sc_y.inverse_transform(y_pred_scaled)
print(y_pred)
y_pred = np.array([y_train.mean() for _ in range(len(y_test))]).reshape(-1, 1)
from sklearn.metrics import mean_squared_error
print(f"MSE: {mean_squared_error(y_test, y_pred)}")













